<a href="https://colab.research.google.com/github/UiinKim/Music_Generation/blob/main/LSTM_MusicGeneration_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from music21 import converter, instrument, note, chord
import glob

notes=[]
for file in glob.glob('C:/Users/UiinKim/MusicGeneration/Classical-Piano-Composer-master/midi_songs/*.mid'):
    midi=converter.parse(file) #converter은 음악을 로드한다. parse를 사용하여 파일의 item을 파싱하고 stream에 넣어준다
    notes_to_parse=None
    try:
        parts=instrument.partitionByInstrument(midi)#단일 stream, score, multi part 구조인 경우에 각각 악기별로 partition을 나누고 다른 파트들을 하나로 합쳐준다.
    except TypeError:
        print('## 1 {} file occur error.'.format(file))
    if parts:
        print('## 2 {} file has instrument parts'.format(file))
        notes_to_parse=parts.parts[0].recurse()
    else:
        print('## 3 {} file has notes in a flat structure'.format(file))
        notes_to_parse=midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch)) #문자열로 note 추가한다
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n)for n in element.normalOrder)) #문자열로 코드 추가

In [ ]:
import numpy as np
from tensorflow.python.keras.utils import np_utils
sequence_length=100
#곡에 있는 모든 계이름의 이름을 pitchnames 변수에 저장 -> set과 sorted로 사전 만들기 준비

pitchnames=sorted(set(item for item in notes))
n_vocab=len(pitchnames)

#음표사전을 만든다
note_to_int=dict((note, number) for number, note in enumerate(pitchnames))
network_input=[]
network_output=[]

#입력 시퀀스 만들기
for i in range(0, len(notes)-sequence_length, 1):
    sequence_in=notes[i:i+sequence_length]
    sequence_out=notes[i+sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns=len(network_input)

#데이터 입력 형태를 LSTM 레이어에 알맞게 변경
network_input=np.reshape(network_input, (n_patterns, sequence_length,1))
#n_pattenrs= batch_size, sequence_length=input_dim, 1=input_length 로 재배치

#입력값을 normalizing(정규화)
network_input=network_input/float(n_vocab) #note_input을 음표사전의 길이로 나눈다
network_output=np_utils.to_categorical(network_output) #one-hot encoding 10진->2진

ValueError: ignored

In [ ]:
from keras.models import Sequenial
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

model=Sequential()
model.add(LSTM(256, input_shape=(network_input[1], network_input[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(245))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

ImportError: ignored